In [14]:
import socket
import json
from datetime import datetime
from ultralytics import YOLO
import cv2
import csv
import folium


In [15]:
def parse_gps_data(gps_data_json):
    """
    Parse the incoming JSON string to extract GPS information.
    """
    try:
        # Ensure the data is a string
        if not isinstance(gps_data_json, str):
            gps_data_json = gps_data_json.decode("ascii", errors="replace")
        
        # Parse JSON string
        gps_data = json.loads(gps_data_json)
        latitude = gps_data.get("latitude")
        longitude = gps_data.get("longitude")
        accuracy = gps_data.get("accuracy")
        altitude = gps_data.get("altitude")
        timestamp = gps_data.get("timestamp")

        print(f"Parsed GPS Data: Latitude={latitude}, Longitude={longitude}, Accuracy={accuracy}, Altitude={altitude}")
        return latitude, longitude, accuracy, altitude, timestamp
    except json.JSONDecodeError as e:
        print(f"Error parsing GPS data: {e}")
        return None, None, None, None, None
    except AttributeError as e:
        print(f"Error: {e}")
        return None, None, None, None, None


In [16]:
def get_gps_coordinates_from_json(ip, port):
    """
    Connect to GPS server and receive JSON-formatted GPS data.
    """
    try:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.connect((ip, port))
            print("Connected to GPS server. Receiving data...")
            while True:
                data = s.recv(1024)  # Receive raw data (bytes)
                latitude, longitude, accuracy, altitude, timestamp = parse_gps_data(data)
                if latitude is not None and longitude is not None:
                    return latitude, longitude, accuracy, altitude, timestamp
    except Exception as e:
        print(f"Error: {e}")
        return None, None, None, None, None


In [17]:
# Initialize YOLO model
model = YOLO("/home/shashank/Desktop/Mini_project /runs/detect/pothole_detection9/weights/best.pt")  

def detect_potholes(image_path):
    """
    Run the YOLOv8 model to detect potholes in an image.
    """
    results = model(image_path)
    detections = []
    
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()  
            confidence = box.conf.cpu().numpy()        
            detections.append((x1, y1, x2, y2, confidence))
    return detections


In [18]:
def annotate_image(image_path, detections, gps_coords):
    """
    Annotate the image with bounding boxes, confidence scores, and GPS coordinates.
    """
    frame = cv2.imread(image_path)
    latitude, longitude = gps_coords

    for (x1, y1, x2, y2, confidence) in detections:
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(frame, f"Pothole ({latitude:.6f}, {longitude:.6f})",
                    (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # Save the annotated image
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_path = f"pothole_detected_{timestamp}.jpg"
    cv2.imwrite(output_path, frame)
    print(f"Annotated image saved as {output_path}")
    return output_path


In [24]:
CSV_FILE = "pothole_detections.csv"

def log_detection_to_csv(latitude, longitude, confidence, timestamp, image_path, accuracy=None, altitude=None):
    """
    Logs detection details to CSV.
    """
    with open(CSV_FILE, "a", newline="") as file:
        writer = csv.writer(file)
        writer.writerow([latitude, longitude, confidence, timestamp, accuracy, altitude, image_path])  # Ensure all 7 values are written



In [28]:
import csv
import folium

def visualize_potholes_on_map(csv_file):
    """
    Create an HTML map showing all detected pothole locations.
    """
    data = []
    with open(csv_file, "r") as file:
        reader = csv.reader(file)
        for row in reader:
            print(f"Row: {row}")  # Debugging line
            data.append(row)

    if data:
        map_center = [float(data[0][0]), float(data[0][1])]
        m = folium.Map(location=map_center, zoom_start=15)

        for row in data:
            if len(row) < 7:  # Check for missing values
                print(f"Skipping invalid row: {row}")  # Debugging message
                continue  # Skip this row
            
            latitude, longitude, confidence, timestamp, accuracy, altitude, image_path = row
            popup_text = f"Confidence: {confidence}<br>Timestamp: {timestamp}<br>Accuracy: {accuracy}m<br>Altitude: {altitude}m<br>Image: {image_path}"
            folium.Marker([float(latitude), float(longitude)], popup=popup_text).add_to(m)

        m.save("pothole_map.html")
        print("Map saved as pothole_map.html")
    else:
        print("No data to visualize.")


In [29]:
if __name__ == "__main__":
    # Replace with your GPS sharing app's IP and port
    GPS_IP = "192.168.1.3"  # Replace with your phone's IP address
    GPS_PORT = 8080           # Replace with the app's port
    IMAGE_PATH = "/home/shashank/Desktop/Mini_project /banglore_potholejpg.jpeg"  # Replace with your test image path

    # Step 1: Get GPS coordinates
    latitude, longitude, accuracy, altitude, timestamp = get_gps_coordinates_from_json(GPS_IP, GPS_PORT)
    if latitude is None or longitude is None:
        print("Unable to fetch GPS coordinates.")
    else:
        print(f"GPS Coordinates: Latitude={latitude}, Longitude={longitude}, Accuracy={accuracy}, Altitude={altitude}")

        # Step 2: Run pothole detection
        detections = detect_potholes(IMAGE_PATH)

        # Step 3: Annotate image with detections and GPS coordinates
        annotated_image_path = annotate_image(IMAGE_PATH, detections, (latitude, longitude))

        # Step 4: Log detections to CSV
        for (x1, y1, x2, y2, confidence) in detections:
            log_detection_to_csv(latitude, longitude, confidence, datetime.now().strftime("%Y-%m-%d %H:%M:%S"), annotated_image_path, accuracy, altitude)

        # Step 5: Visualize logged detections on a map
        visualize_potholes_on_map(CSV_FILE)


Connected to GPS server. Receiving data...
Error: 'int' object has no attribute 'get'
Error: 'int' object has no attribute 'get'
Error: 'int' object has no attribute 'get'
Error: 'int' object has no attribute 'get'
Error: 'int' object has no attribute 'get'
Parsed GPS Data: Latitude=12.9170451, Longitude=77.517333, Accuracy=500.0, Altitude=762.0999755859375
GPS Coordinates: Latitude=12.9170451, Longitude=77.517333, Accuracy=500.0, Altitude=762.0999755859375

image 1/1 /home/shashank/Desktop/Mini_project /banglore_potholejpg.jpeg: 480x640 3 potholes, 84.3ms
Speed: 3.0ms preprocess, 84.3ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)
Annotated image saved as pothole_detected_20250214_103810.jpg
Row: ['12.9156634', '77.5190908', '[    0.86814]', '2025-01-15 15:50:15', '10.083000183105469', '761.3999633789062', 'pothole_detected_20250115_155015.jpg']
Row: ['12.9156634', '77.5190908', '[    0.73183]', '2025-01-15 15:50:15', '10.083000183105469', '761.3999633789062', 'pot